# The Data Optimization Dilemma

## 1. What is the Issue?
We are currently trapped in a cycle of **Reactive Maintenance**.
* **The Bottleneck:** As data grows, query performance degrades (SLA breaches).
* **The Burden:** Engineers must manually intervene to analyze skew, identify slow queries, and re-engineer table structures.
* **The Risk:** Traditional fixes like **Repartitioning** require full table rewrites (`overwriteSchema`), which are expensive, risky, and lock the table for long periods.
* **The Complexity:** We are forced to maintain complex "Auto-Tuner" scripts that rely on fragile regex parsing of query history just to guess the right configuration.

## 2. What We Hope We Could Do
Our ideal state is an **Autonomous Data Platform**:
* **Self-Optimizing:** The system should automatically "learn" which columns are frequently accessed without us writing regex parsers.
* **Flexible:** We should be able to change how data is clustered (e.g., from `date` to `region`) instantly, without rewriting 10 years of history.
* **Resilient:** The system should handle data skew automatically, preventing "straggler" tasks without manual "salting" techniques.
* **Zero-Touch:** We want to define *what* we want (fast queries on `symbol`), not *how* to store it (file layout).

## 3. What We Can Do Today (The "Hard Way")
Currently, to achieve performance, we are forced to build complex "Rube Goldberg" machines:
1.  **Query Mining:** We scrape `system.query.history` to find frequent columns.
2.  **Rigid Partitioning:** We hard-code a physical directory structure (e.g., `/date=2024-01-01/`). If access patterns change, this structure becomes a prison.
3.  **Scheduled Heavy Lifting:** We run expensive `OPTIMIZE ZORDER BY` jobs that fight for resources.
4.  **Manual Skew Fixes:** When a partition gets too big, we manually intervene to split it.

## 4. How We Will Get to Our Hope: Liquid Clustering
We bridge this gap by abandoning physical directory partitioning entirely and adopting **Delta Liquid Clustering**.

* **The Shift:** Instead of rigid physical folders, Liquid Clustering uses a flexible, dynamic index managed by Delta Lake.
* **The Solution:**
    * **No Rewrites:** We can redefine clustering keys on the fly using `ALTER TABLE`.
    * **Skew Handling:** It automatically detects skew and splits data into appropriate sizes without user intervention.
    * **Simplicity:** It replaces our entire 100-line "AutoTuner" script with a single line of configuration.

> **Next Step:** We will demonstrate how replacing `PARTITIONED BY` with `CLUSTER BY` eliminates the need for the complex Python logic we just wrote.

In [0]:
from pyspark.sql.functions import col, count, lower, explode, split, expr
import re

# --- CONFIGURATION ---
TARGET_TABLE = "silver_audit"

class TableAutoTuner:
    def __init__(self, spark, table_name):
        self.spark = spark
        self.table_name = table_name
        self.valid_columns = []
        self.candidates = []

    def _get_valid_columns(self):
        """
        Fetches the actual schema of the table to validate extracted text.
        """
        print(f"[INFO] Fetching schema for {self.table_name}...")
        try:
            # Get list of actual column names (e.g., ['symbol', 'date', 'eps'])
            self.valid_columns = self.spark.read.table(self.table_name).columns
            print(f"[INFO] Valid Schema Columns: {self.valid_columns}")
        except Exception as e:
            print(f"[ERROR] Could not read table schema: {str(e)}")

    def _extract_columns_from_history(self):
        """
        Parses system history and finds which valid columns are used most often.
        """
        print(f"[INFO] Mining query history for {self.table_name}...")
        
        # 1. Fetch raw SQL statements involving this table
        history_df = self.spark.sql(f"""
            SELECT statement_text 
            FROM system.query.history 
            WHERE statement_text LIKE '%{self.table_name}%'
              AND statement_type = 'SELECT'
              AND start_time > date_add(now(), -30)
        """)

        if history_df.count() == 0:
            print("[WARN] No history found. Cannot optimize.")
            return

        # 2. Tokenize the SQL text
        # We split by non-alphanumeric characters to isolate words
        words_df = (history_df
                    .select(explode(split(lower(col("statement_text")), "[^a-z0-9_]")).alias("word"))
                    .filter(col("word") != "")
                   )

        # 3. Filter for words that are ACTUAL table columns
        # This prevents "WHERE" or "SELECT" from being treated as column names
        # AND prevents errors like 'company_symbol' vs 'symbol'
        valid_cols_broadcast = [c.lower() for c in self.valid_columns]
        
        matched_cols_df = (words_df
                           .filter(col("word").isin(valid_cols_broadcast))
                           .groupBy("word")
                           .agg(count("*").alias("frequency"))
                           .sort(col("frequency").desc())
                           .limit(5) # Top 5 used columns
                          )

        print("[INFO] Top detected columns from history:")
        matched_cols_df.show()
        
        # Convert to list for further processing
        self.candidates = [row["word"] for row in matched_cols_df.collect()]

    def _analyze_cardinality_and_recommend(self):
        """
        Decides between Partitioning vs. Z-Order based on unique values.
        """
        if not self.candidates:
            print("[INFO] No valid usage patterns found.")
            return

        print("\n--- Optimization Recommendations ---")
        
        for col_name in self.candidates:
            try:
                # Check how many unique values exist
                distinct_count = self.spark.table(self.table_name).select(col_name).distinct().count()
                
                print(f"Column: '{col_name}' | Distinct Values: {distinct_count}")
                
                # LOGIC MATRIX
                if distinct_count > 10000:
                    print(f"  -> RECOMMENDATION: **Z-ORDER** (High Cardinality).")
                    print(f"     Why: Too many unique values for partitioning. Z-Order will skip data effectively.")
                    
                elif 50 < distinct_count <= 10000:
                    print(f"  -> RECOMMENDATION: **Z-ORDER** (Medium Cardinality).")
                    print(f"     Why: Safe for Z-Order. Risky for partitioning (potential small files).")
                    
                else: # < 50
                    print(f"  -> RECOMMENDATION: **PARTITION** (Low Cardinality).")
                    print(f"     Why: Few unique values. Good candidate for physical folder separation.")
                    
                print("-" * 50)

            except Exception as e:
                print(f"[ERROR] failed to analyze {col_name}: {str(e)}")

    def run(self):
        print(f"--- Auto-Discovery Tuner for {self.table_name} ---")
        self._get_valid_columns()
        self._extract_columns_from_history()
        self._analyze_cardinality_and_recommend()

# --- EXECUTION ---
tuner = TableAutoTuner(spark, TARGET_TABLE)
tuner.run()